In [ ]:
import pandas as pd

file = pd.read_csv("Combined Data.csv")


file.info()
file.isnull().sum()
file.duplicated().sum()
print(file.columns)
print(file['statement'].describe())
file['status'].nunique()
file['status'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53043 entries, 0 to 53042
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  53043 non-null  int64 
 1   statement   52681 non-null  object
 2   status      53043 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.2+ MB
Index(['Unnamed: 0', 'statement', 'status'], dtype='object')

count                 52681
unique                51073
top       what do you mean?
freq                     22
Name: statement, dtype: object
 



array(['Anxiety', 'Normal', 'Depression', 'Suicidal', 'Stress', 'Bipolar',
       'Personality disorder'], dtype=object)

In [ ]:
import pandas as pd
import spacy
import re
from tqdm import tqdm
tqdm.pandas()
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner", "tagger"])

file = pd.read_csv("Combined Data.csv")

file = file.dropna()

file = file.drop_duplicates(subset=['statement'])

file['statement_'] = file['statement'].astype('string')
assert file['statement_'].dtype == 'string'

file['statement_'] = file['statement_'].apply(lambda x: re.sub(r'[^\w\s]', '', x))

file['statement_'] = file['statement_'].str.lower()
assert file['statement_'].apply(lambda x : x == x.lower()).all()

texts = file["statement"].tolist()
file["lemmatized"] = [process_text(text) for text in tqdm(texts)]


def process_text(text):
    doc = nlp(text)
    return " ".join([token.lemma_ for token in doc])

file.info()
file.sample(2)


In [ ]:
from google.colab import files
file[['statement_', 'lemmatized', 'status']].to_csv("cleaned_sentiment.csv", index=False)
files.download("cleaned_sentiment.csv")

In [ ]:
!pip install -U sentence-transformers



In [ ]:
from sentence_transformers import SentenceTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

file = pd.read_csv('cleaned_sentiment.csv').dropna()

bert_model = SentenceTransformer('all-MiniLM-L6-v2')

X_embeddings = bert_model.encode(file['lemmatized'].tolist(),batch_size=256, show_progress_bar=True, convert_to_numpy=True)
pca = PCA(n_components=100)
X_clean = pca.fit_transform(X_embeddings)
X_train, X_test, y_train, y_test = train_test_split(X_embeddings, file['status'], test_size=0.2, random_state=42)


model = LogisticRegression(max_iter=1000, class_weight='balanced', C=0.1)
model.fit(X_train, y_train)
y_predict = model.predict(X_test)


print(f" accuracy : {accuracy_score(y_test, y_predict)}")
print(classification_report(y_test, y_predict))

In [ ]:
!pip install joblib
!pip install gradio


In [ ]:
import joblib

joblib.dump(model, "emotion_classifier.pkl")

In [ ]:
import re
import spacy
from sentence_transformers import SentenceTransformer
import joblib

bert_model = SentenceTransformer('all-MiniLM-L6-v2')
model = joblib.load("emotion_classifier.pkl")
nlp = spacy.load("en_core_web_sm", disable=["parser", "ner", "tagger"])

def yay(text):
  text = text.lower()
  text = re.sub(r'[^\w\s]', '', text)
  doc = nlp(text)
  lemma = [' '.join([token.lemma_ for token in doc])]
  embed = bert_model.encode(lemma, convert_to_numpy=True)
  predict = model.predict(embed)[0]
  proba = model.predict_proba(embed)[0]
  return predict

advices = [
    ["depression", "You're not alone. It's important to talk to someone you trust and seek support from a mental health professional."],
    ["anxiety", "Take a deep breath. Focus on grounding yourself through slow breathing or calming activities like journaling or music."],
    ["suicidal", "Please talk to someone immediately. You're important and your life matters. Reach out to a crisis line or a trusted person."],
    ["stress", "Give yourself a break. Even small pauses, a walk, or calming music can help reduce tension. Take care of your body and mind."],
    ["bipolar", "Mood shifts are not your fault. Keeping a regular routine and getting professional support helps you stay stable."],
    ["personality disorder", "You deserve understanding and support. Therapy and safe relationships can help you grow and feel more grounded."],
    ["neutral", "I'm here to listen. You can share anything you feel, at your own pace."]
]


while True :
  user = input("You can express yourself freely it's a safe space press exit if you want to exit  ")

  if user.lower() == "exit":
    break
  print(f"your feeling rn is : {yay(user)}")
  for y in advices :
    if yay(user) == y[0] :
      print(y[1])
